## Import libraries

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Add path to custom modules
import sys
import os
os.environ['PATH'] = os.environ['PATH'] + ':/Library/TeX/texbin'
this_path = os.path.abspath('') 
parent_dir = os.path.dirname(this_path)  
sys.path.append(parent_dir)

# Import custom modules
from Modules import Parameters, DataPreprocessing as DP, Classification as CL, CustomFunctions as CF

## Parameters

In [ ]:
## Multivariate sets
immunecells_set = Parameters.immunecells_set
FC_set = Parameters.FC_set
Dem_set = Parameters.Dem_set
CK_set = Parameters.CK_set
BM_set =  Parameters.BM_set
allinput_set = Parameters.allinput_set

## Targets
target_train = Parameters.train_target
target_test = Parameters.test_target

## Minimum NPV
min_NPV_Models = Parameters.min_NPV_Models
min_NPV = Parameters.min_NPV

## Nan masking row-wise
do_nan_masking = Parameters.do_nan_masking
do_nan_masking_univ = Parameters.do_nan_masking_univ
nan_masking = Parameters.nan_masking
do_nan_masking_groupwise = False # False: filter nans independently for each variable

## N samples for average
N_av = Parameters.N_av

## Imputation
imputation_method = Parameters.imputation_method
imputation_method_univ = Parameters.imputation_method_univ

## Standardization
std_method = Parameters.std_method

## PCA % var. threshold
pc_var_th = Parameters.pca_var_threshold

## Preprocessing
do_preprocessing = Parameters.do_preprocessing_univ

## Train-test
test_size = Parameters.test_size

## Paths
path_results = Parameters.path_results
path_figures = Parameters.path_figures
exp_description = Parameters.exp_description
exp_univ_description = Parameters.exp_univ_description
foldername_univ = Parameters.foldername_univ

## Run experiments
run_experiments = True

## Import and prepare datasets

In [ ]:
DataInpatients, DataOutpatients = DP.data_preprocessing()

# Mask data by target
mask = pd.notnull(DataInpatients[target_train].values) & pd.notnull(DataInpatients[target_test].values)
DataInpatients = DataInpatients.loc[mask,:]

## Models dictionary

In [ ]:
models_dict = {}

## LR models
models_dict['LR'] = {}
for variable in allinput_set:
    models_dict['LR'][variable] = [variable]

## Run analysis - multiple splits

In [ ]:
if run_experiments:
    Data = DataInpatients.copy()     
    
    ## Initialize results dictionaries
    Results_N_av = {}
    Ground_Truth_N_av = {}
    for model in models_dict.keys():
        Results_N_av[model] = {}
        Ground_Truth_N_av[model] = {}
        for set_name in models_dict[model].keys():
            Results_N_av[model][set_name] = {'Train':[], 'Test':[], 'Train_value': [], 'Test_value': [], 'Weights':[], 'Bias':[], 'C':[], 'Std_Parameters': []}
            Ground_Truth_N_av[model][set_name] = {'Train':[], 'Test':[]}
            if min_NPV_Models:
                Results_N_av[model][set_name+'_minNPV'] = {'Train':[], 'Test':[], 'Train_value': [], 'Test_value': [], 'Cutoff': []}
                Ground_Truth_N_av[model][set_name+'_minNPV'] = {'Train':[], 'Test':[]}
                 

    ## Sets of variables for nan removal
    groups = [FC_set, CK_set, Dem_set, BM_set]


    ## Run average
    fix_outliers = False
    do_imputation = False
    print('Running average...')
    print('Fix outliers: %s' % fix_outliers)
    print('Do imputation: %s' % do_imputation)
    print('Nan masking groupwise: %s' % do_nan_masking_groupwise)
    print('Do preprocessing: %s' % do_preprocessing)
    for i_split in range(N_av): 
        idx_split = i_split+1
        print('%d/%d' %(idx_split, N_av))
        
        Results = CL.models_prediction(Data=Data.copy(),
                                       test_size=test_size,
                                       models_dict=models_dict,
                                       target_train=target_train, 
                                       target_test=target_test, 
                                       min_NPV=min_NPV,
                                       min_NPV_Models=min_NPV_Models,
                                       standardization=std_method, 
                                       imputation=imputation_method, 
                                       do_nan_masking=do_nan_masking,
                                       do_nan_masking_univ=do_nan_masking_univ,
                                       nan_masking=nan_masking, 
                                       do_nan_masking_groupwise=do_nan_masking_groupwise, 
                                       groups=groups,
                                       do_preprocessing=do_preprocessing, 
                                       fix_outliers=fix_outliers,
                                       do_imputation=do_imputation, 
                                       random_state=idx_split)

        for model in Results_N_av.keys():
            for set_name in Results_N_av[model].keys():
                Ground_Truth_N_av[model][set_name]['Train'].append(Results[model][set_name]['Train_Labels'])
                Ground_Truth_N_av[model][set_name]['Test'].append(Results[model][set_name]['Test_Labels'])
                Results_N_av[model][set_name]['Train_value'].append(Results[model][set_name]['Train_value'])
                Results_N_av[model][set_name]['Test_value'].append(Results[model][set_name]['Test_value'])
                Results_N_av[model][set_name]['Train'].append(Results[model][set_name]['Train'])
                Results_N_av[model][set_name]['Test'].append(Results[model][set_name]['Test'])
                
                if model=='LR' and 'minNPV' in set_name:
                    Results_N_av[model][set_name]['Cutoff'].append(Results[model][set_name]['Cutoff'])

                if model=='LR' and '_minNPV' not in set_name:
                    Results_N_av[model][set_name]['Weights'].append(Results[model][set_name]['Weights'])
                    Results_N_av[model][set_name]['Bias'].append(Results[model][set_name]['Bias'])
                    Results_N_av[model][set_name]['C'].append(Results[model][set_name]['C'])
                    Results_N_av[model][set_name]['Std_Parameters'].append(Results[model][set_name]['Std_Parameters'])

## Compute performanes

In [ ]:
if run_experiments:

    Performances = {}

    Scores = {'roc_auc_score': roc_auc_score,
              'f1': f1_score,
              'recall': recall_score,
              'precision': precision_score,
              'NPV': precision_score,
              'specificity': recall_score,
              'accuracy': accuracy_score}

    for model in Results_N_av.keys():
        Performances[model] = {}

        for set_name in Results_N_av[model].keys():
            #print(model, set_name)
            Performances[model][set_name] = {}
            print(model, set_name)

            for score in Scores.keys():
                Performances[model][set_name][score] = {'Train': [], 'Test': []}
                sc = Scores[score]
                for sample in range(N_av):
                    y_pred_train = Results_N_av[model][set_name]['Train'][sample]
                    y_value_train = Results_N_av[model][set_name]['Train_value'][sample]
                    y_train = Ground_Truth_N_av[model][set_name]['Train'][sample]
                    y_pred_test = Results_N_av[model][set_name]['Test'][sample]
                    y_value_test = Results_N_av[model][set_name]['Test_value'][sample]
                    y_test = Ground_Truth_N_av[model][set_name]['Test'][sample]

                    y_pred_train_1 = 1 - y_pred_train
                    y_train_1 = 1 - y_train
                    y_pred_test_1 = 1 - y_pred_test
                    y_test_1 = 1 - y_test

                    if score=='NPV':
                        if sum(pd.notnull(y_pred_train_1))>1:
                            score_val_train = sc(y_train_1, y_pred_train_1, zero_division=0)
                        else:
                            score_val_train = np.nan

                        if sum(pd.notnull(y_pred_test_1))>1:
                            score_val_test = sc(y_test_1, y_pred_test_1, zero_division=0)
                        else:
                            score_val_test = np.nan

                    elif score=='specificity':
                        if sum(pd.notnull(y_pred_train_1))>1:
                            score_val_train = sc(y_train_1, y_pred_train_1)
                        else:
                            score_val_train = np.nan

                        if sum(pd.notnull(y_pred_test_1))>1:
                            score_val_test = sc(y_test_1, y_pred_test_1)
                        else:
                            score_val_test = np.nan

                    elif score=='roc_auc_score':
                        if sum(pd.notnull(y_pred_train))>1:
                            score_val_train = sc(y_train, y_value_train)
                        else:
                            score_val_train = np.nan

                        if sum(pd.notnull(y_pred_test))>1:
                            score_val_test = sc(y_test, y_value_test)
                        else:
                            score_val_test = np.nan

                    elif score=='precision':
                        if sum(pd.notnull(y_pred_train))>1:
                            score_val_train = sc(y_train, y_pred_train, zero_division=0)
                        else:
                            score_val_train = np.nan

                        if sum(pd.notnull(y_pred_test))>1:
                            score_val_test = sc(y_test, y_pred_test, zero_division=0)
                        else:
                            score_val_test = np.nan

                    else:
                        if sum(pd.notnull(y_pred_train))>1:
                            score_val_train = sc(y_train, y_pred_train)
                        else:
                            score_val_train = np.nan

                        if sum(pd.notnull(y_pred_test))>1:
                            score_val_test = sc(y_test, y_pred_test)
                        else:
                            score_val_test = np.nan

                    Performances[model][set_name][score]['Train'].append(score_val_train)
                    Performances[model][set_name][score]['Test'].append(score_val_test)

### Average Performances

In [ ]:
if run_experiments:

    # Train performances
    test_or_train = 'Train'
    Performances_mean_ci_train = {}
    Performances_mean_ci_train_0 = {}
    notnull_prop = 0.9
    normality_pval_th = 0.001

    for score in Scores:
        Performances_mean_ci_train[score] = {'Mean':[], 'CI_lower':[], 'CI_upper':[]}
        Performances_mean_ci_train_0[score] = {'Mean':[], 'CI_lower':[], 'CI_upper':[]}

    for model in Performances.keys():
        for set_name in Performances[model].keys():
            for score in Performances[model][set_name].keys():

                v = np.array(Performances[model][set_name][score][test_or_train])
                mask = pd.notnull(v)
                if sum(mask)>notnull_prop*len(mask):
                    v = v[pd.notnull(v)]
                    # Normality test
                    normality_pval = st.normaltest(v, nan_policy='omit')[1]
                    if normality_pval>normality_pval_th:
                        ci_lower, ci_upper = st.t.interval(alpha=0.95, df=len(v)-1, loc=np.mean(v), scale=st.sem(v))
                        score_mean = np.mean(v)
                    else:
                        ci_lower, ci_upper, score_mean = CF.bootstrap_ci_mean(v)
                else:
                    ci_upper = 0
                    ci_lower = 0 
                    score_mean = 0

                if '_minNPV' not in set_name:
                    Performances_mean_ci_train[score]['Mean'].append(score_mean)
                    Performances_mean_ci_train[score]['CI_lower'].append(ci_lower)
                    Performances_mean_ci_train[score]['CI_upper'].append(ci_upper)
                else:
                    Performances_mean_ci_train_0[score]['Mean'].append(score_mean)
                    Performances_mean_ci_train_0[score]['CI_lower'].append(ci_lower)
                    Performances_mean_ci_train_0[score]['CI_upper'].append(ci_upper)


    # Test performances
    test_or_train = 'Test'
    Performances_mean_ci_test = {}
    Performances_mean_ci_test_0 = {}

    for score in Scores:
        Performances_mean_ci_test[score] = {'Mean':[], 'CI_lower':[], 'CI_upper':[]}
        Performances_mean_ci_test_0[score] = {'Mean':[], 'CI_lower':[], 'CI_upper':[]}

    for model in Performances.keys():
        for set_name in Performances[model].keys():
            for score in Performances[model][set_name].keys():

                v = np.array(Performances[model][set_name][score][test_or_train])
                mask = pd.notnull(v)
                if sum(mask)>notnull_prop*len(mask):
                    v = v[pd.notnull(v)]
                    # Normality test
                    normality_pval = st.normaltest(v, nan_policy='omit')[1]
                    if normality_pval>normality_pval_th:
                        ci_lower, ci_upper = st.t.interval(alpha=0.95, df=len(v)-1, loc=np.mean(v), scale=st.sem(v))
                        score_mean = np.mean(v)
                    else:
                        ci_lower, ci_upper, score_mean = CF.bootstrap_ci_mean(v)
                else:
                    ci_upper = 0
                    ci_lower = 0 
                    score_mean = 0

                if '_minNPV' not in set_name:
                    Performances_mean_ci_test[score]['Mean'].append(score_mean)
                    Performances_mean_ci_test[score]['CI_lower'].append(ci_lower)
                    Performances_mean_ci_test[score]['CI_upper'].append(ci_upper)
                else:
                    Performances_mean_ci_test_0[score]['Mean'].append(score_mean)
                    Performances_mean_ci_test_0[score]['CI_lower'].append(ci_lower)
                    Performances_mean_ci_test_0[score]['CI_upper'].append(ci_upper)

### Average cutoff thresholds

In [ ]:
if run_experiments:

    cutoff_mean_ci_train = {'Mean':[], 'CI_lower':[], 'CI_upper':[]}

    test_or_train = 'Train'

    for model in Results_N_av.keys():
        for set_name in Results_N_av[model].keys():

            if 'minNPV' in set_name:
                v = np.array(Results_N_av[model][set_name]['Cutoff'])
                mask = pd.notnull(v)
                if sum(mask)>notnull_prop*len(mask):
                    v = v[pd.notnull(v)]
                    # Normality test
                    normality_pval = st.normaltest(v, nan_policy='omit')[1]
                    if normality_pval>normality_pval_th:
                        ci_lower, ci_upper = st.t.interval(alpha=0.95, df=len(v)-1, loc=np.mean(v), scale=st.sem(v))
                        cutoff_mean = np.mean(v)
                    else:
                        ci_lower, ci_upper, cutoff_mean = CF.bootstrap_ci_mean(v)
                else:
                    ci_upper = 0
                    ci_lower = 0 
                    cutoff_mean = 0

                cutoff_mean_ci_train['Mean'].append(cutoff_mean)
                cutoff_mean_ci_train['CI_lower'].append(ci_lower)
                cutoff_mean_ci_train['CI_upper'].append(ci_upper)

### Export results

In [ ]:
if run_experiments:
    # Create directory
    try:
        os.mkdir(path_results+foldername_univ)
    except OSError:
        print ("Creation of the directory failed")
    else:
        print ("Successfully created the directory")

    path_export = path_results+foldername_univ

    # Export 

    # Tuples here are: (name_score, mean) (name_score, CI_lower) (name_score, CI_upper) #
    my_tuples = [(score, val) for score in Scores for val in ['Mean', 'CI_lower', 'CI_upper']]
    new_columns = pd.MultiIndex.from_tuples(my_tuples)
    index = []
    for model in models_dict.keys():
        index.extend([model+': '+name for name in models_dict[model].keys()])
    n_models = len(index)
    values = np.array([np.array(Performances_mean_ci_train[score][val]) for score in Scores for val in ['Mean', 'CI_lower', 'CI_upper']]).transpose()
    df_results = pd.DataFrame(values, columns=new_columns, index=index)
    filename = 'performances_train.xlsx'
    df_results.to_excel(path_export+filename, engine='xlsxwriter')

    if min_NPV_Models:
        values_0 = np.array([np.array(Performances_mean_ci_train_0[score][val]) for score in Scores for val in ['Mean', 'CI_lower', 'CI_upper']]).transpose()
        df_results_0 = pd.DataFrame(values_0, columns=new_columns, index=index)
        filename_0 = 'performances_minNPV_train.xlsx'
        df_results_0.to_excel(path_export+filename_0, engine='xlsxwriter')


    # Tuples here are: (name_score, mean) (name_score, CI_lower) (name_score, CI_upper) #
    values = np.array([np.array(Performances_mean_ci_test[score][val]) for score in Scores for val in ['Mean', 'CI_lower', 'CI_upper']]).transpose()
    df_results = pd.DataFrame(values, columns=new_columns, index=index)
    filename = 'performances_test.xlsx'
    df_results.to_excel(path_export+filename, engine='xlsxwriter')

    if min_NPV_Models:
        values_0 = np.array([np.array(Performances_mean_ci_test_0[score][val]) for score in Scores for val in ['Mean', 'CI_lower', 'CI_upper']]).transpose()
        df_results_0 = pd.DataFrame(values_0, columns=new_columns, index=index)
        filename_0 = 'performances_minNPV_test.xlsx'
        df_results_0.to_excel(path_export+filename_0, engine='xlsxwriter')

    # Export cutoffs
    if min_NPV_Models:
        models_list = [name for name in Results_N_av[model].keys() if '_minNPV' not in name]
        df_cutoff = pd.DataFrame(cutoff_mean_ci_train, index=models_list)
        filename_0 = 'cutoffs_minNPV_test.xlsx'
        df_cutoff.to_excel(path_export+filename_0, engine='xlsxwriter')

    # Export models description
    models_description = {}
    for model in models_dict.keys():
        for name in models_dict[model].keys():
            description = ' # '.join(models_dict[model][name])
            models_description[model+'#'+name] = description

    filename = 'models_description.xlsx'
    pd.Series(models_description).to_excel(path_export+filename)

## Figures

In [ ]:
# Color scheme
color_0 = Parameters.dark_blue
color_1 = Parameters.red_purp
color_control = Parameters.green

# Parameters
fontsize = 9
ylabelsize = 7
xlabelsize = 7 
tex = True
axes_lines_w = 0.5
lines_w = 0.5

### Histogram

In [ ]:
model_name = 'RTE/uL' # 'RTE/uL', 'IP10', 'proADM' 

Data_train = DataInpatients.copy()
Data_test = DataOutpatients.copy() # Data_Control
if model_name not in immunecells_set:
    Data_test = DataInpatients.copy() 
    
print(Data_train.shape, Data_test.shape)

In [ ]:
random_state = None
columns = models_dict['LR'][model_name]
fix_outliers = False
do_imputation = True
groups = [FC_set, CK_set, Dem_set, BM_set]
#columns = list(np.random.permutation(columns))

results, MLR, prep_data = CL.LR_model_results(Data=Data_train,
                                              Data_test=Data_test,
                                              features=columns, 
                                              set_name=model_name, 
                                              target_train=target_train, 
                                              target_test=target_test, 
                                              test_size=test_size,
                                              hyperp_dict={}, 
                                              do_preprocessing=do_preprocessing, 
                                              fix_outliers=fix_outliers, 
                                              do_imputation=do_imputation, 
                                              imputation=imputation_method, 
                                              pca_var_threshold=pc_var_th, 
                                              standardization=std_method, 
                                              min_NPV_Models=min_NPV_Models,
                                              min_NPV=min_NPV, 
                                              groups=groups, 
                                              do_nan_masking=do_nan_masking, 
                                              do_nan_masking_groupwise=do_nan_masking_groupwise, 
                                              do_nan_masking_univ=do_nan_masking_univ, 
                                              nan_masking=nan_masking,
                                              return_model=True, 
                                              return_data=True)

In [ ]:
# Prepare data histogram
Data_X = prep_data['Train']
y_train = results[model_name]['Train_Labels']
y_pred = results[model_name]['Train']
value_pred = results[model_name]['Train_value']
std = StandardScaler()
value_pred = std.fit_transform(value_pred.reshape(-1, 1)).reshape(-1,)
x_test = results[model_name]['Test_value']
x_test = std.transform(x_test.reshape(-1, 1)).reshape(-1,)
mask_0 = y_train==0
mask_1 = y_train==1
x0 = value_pred[mask_0]
x1 = value_pred[mask_1]

In [ ]:
magnification = 0.85
ratio = 2.4/3.7
CF.SetPlotParams(magnification=magnification, ratio=ratio, 
                        fontsize=fontsize, ylabelsize=ylabelsize, xlabelsize=xlabelsize, 
                        tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w)
mec = 'white'
ms = 9
mew = 0.15

fig, ax = plt.subplots()

y_target = y_train
th = CL.best_threshold_class0(y_pred=y_pred, value_pred=value_pred, y_target=y_train, min_NPV=0.96, fixed_threshold=False)
y_pred = np.zeros_like(y_target)
y_pred[value_pred>=th] = 1
y_pred_1 = 1 - y_pred
y_target_1 = 1 - y_target
x_min = min([np.nanmin(x0), np.nanmin(x1), np.nanmin(x_test)])
x_max = max([np.nanmax(x0), np.nanmax(x1), np.nanmax(x_test)])

if model_name in immunecells_set:
    x_control = x_test
    print('control: %d%%' % (round(100*np.sum(x_control<th)/len(x_control))))

bins = np.linspace(x_min-0.2, x_max+0.2, 14)

color = color_0
density = False
ax.hist(x0, bins,
        alpha=0.15,
        histtype='stepfilled',
        color=color,
        density=density)
ax.hist(x0, bins,
        alpha=1., 
        histtype='step',
        edgecolor=color,
        density=density,
        facecolor='None', 
        label='no event')

bins = np.linspace(x_min-0.2, x_max+0.2, 13)

if model_name in immunecells_set:
    color = color_control
    ax.hist(x_control, bins,
            alpha=0.2,
            histtype='stepfilled',
            color=color,
            density=density,
            label='control')
    ax.hist(x_control, bins,
            alpha=1.,
            histtype='step',
            edgecolor=color,
            density=density,
            facecolor='None')

color = color_1
ax.hist(x1, bins,
        alpha=0.2,
        histtype='stepfilled',
        color=color,
        density=density,
        label='event')
ax.hist(x1, bins,
        alpha=1., 
        histtype='step',
        edgecolor=color,
        density=density,
        facecolor='None')

color_boundary = CF.lighten_color(Parameters.grey_1, 0.6)
ax.axvline(th, ls='--', lw=0.7, color='black') 

ax.title.set_text('Score Histogram')
ax.set_xlabel('%s score' % (CF.change_names([model_name])[0]))
ax.set_ylabel('n. patients')
if density:
    ax.set_ylabel('patients (density)')


if model_name=='RTE/uL':
        ax.set_yticks([0., 45., 90])
        ax.set_ylim([0, 90])
if model_name=='proADM':
        ax.set_yticks([0., 40., 80.])
        ax.set_ylim([0, 80])
if model_name=='IP10':
        ax.set_yticks([0., 20., 40.])
        ax.set_ylim([0, 40])


leg = ax.legend(loc='center left', bbox_to_anchor=(0.98, 0.5), frameon=False) #legend(loc=0, frameon=False)
for lh in leg.legendHandles: 
    lh.set_alpha(1)
    lh.set_edgecolor('white')

filename = 'HistogramLR#%s' % (CF.change_names([model_name])[0])
filename = filename + exp_description + exp_univ_description + '.pdf'
saving_str = path_figures + filename
#plt.savefig(saving_str, bbox_inches='tight')